# HTTP request methods

Come detto, il protocollo HTTP prevede una serie di [messaggi di richiesta](https://it.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Messaggio_di_richiesta) che il client può inviare al server. In particolare i più noti sono

- GET
- POST
- PUT
- DELETE
- ...

**PER APPROFINDIRE:** https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods

Inizialmente vedre o prima il metodo GET e poi il metodo POST.

## Esempi di richiesta GET

Se nel decoratore `@app.route()` l'argomento `methods` è omesso, allora di default è GET.

Se viene richiesta una pagina web, questo avviene di solito tramite una richiesta GET.

> GET significa "ottenere", dunque quando vogliamo *ottenere* una risorsa, come un file o una pagina web, la richiesta appropriata è proprio GET!

### GET Standatd

- `https://mio-sito.org/hello`

```python
@app.route('/hello')  # Sottinteso GET (e solo GET)
def hello():
    return 'Hello, World!'
```

### GET con variabili nell'URL

- `https://mio-sito.org/wiki/Albert_Einstein`
- `https://mio-sito.org/wiki/Python`


```python
@app.route('/wiki/<name>')  # Sottinteso GET (e solo GET)
def wiki(name):
    dati_pagina = ottieni_pagina_wiki(escape(name))  # (ipotetica funzione)
    return render_template('wiki_voce.html', data=dati_pagina)
```

### GET con variabili "tipizzate" nell'URL

- `https://mio-sito.org/post/872`
- `https://mio-sito.org/post/23457`


```python
@app.route('/post/<int:post_id>')
def show_post(post_id):
    dati_post = ottieni_pagina_wiki(post_id)  # (ipotetica funzione)
    return render_template('post.html', data=data)

```

### GET con parametri

Se nell'URL della richiesta vengono inseriti dei parametri con la sintassi `?key=value`, possiamo accedere ad essi tramite `request.args`, il quale si comporta come un dizionario.

- `https://mio-sito.org/ricerca?livello=intermedio`
- `https://mio-sito.org/ricerca?categoria=informatica`

```python
@app.route('/ricerca')  # Sottinteso GET (e solo GET)
def ricerca():
    categoria = request.args.get('categoria')  # in questo caso il default è None
    livello = request.args.get('livello', default='tutti')  # con default personalizzato
    return esegui_ricerca(category=categoria, level=livello)  # (ipotetica funzione)
```

> NOTA: Usiamo `.get()` perché è più "sicuro" in quanto non genera errori se la chiave non è presente sun `request.args`.

## Esempi di richiesta POST

Nel metodo POST, i parametri non vengono inclusi nell'URL come avviene con il metodo GET. 

Dati e parametri di una richiesta POST sono inclusi nel corpo della richiesta stessa. Questo permette di inviare quantità di dati molto più grandi rispetto al GET, poiché non sono limitati dalla lunghezza massima dell'URL. 

> **IMPORTANTE:** Poiché i dati non appaiono nell'URL (come avviene per il GET), il metodo POST è più sicuro per il trasferimento di informazioni sensibili, come password o dati personali.

Nel decoratore `@app.route()` possiamo passare un argomento `methods` per esplicitare a quali metodi la funzione deve rispondere.


```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':  # Se la richiesta è di tipo POST
        ...  # Gestione della richiesta POST
    else:  # Se non è po POST, allora è sicuramente GET
        ...  # Gestione della richiesta GET
```

**NOTA:** Il metodo  POST lo vedremo la prossima lezione.

# Template Jinja

Flask ci mette a disposizione il motore di templating Jinja2.

In pratica un motore di templating serve per interpolare (inserire) dati all'interno di un documento in modo dinamico.

Per usare Jinja dovremo imparare un nuovo "mini linguaggio", ma che per nostra fortuna è molto simile a Python.

La documentazione ufficiale di Jinja la trovate qua:

- [Jinja2 Template Designer Documentation](https://jinja.palletsprojects.com/en/3.1.x/templates/)

## Creazione di un template Jinja

Quello che segue è un semplice template che mostra un heading `<h1>` diverso a seconda che una condizione si verifichi o no.

In particolare, si chiede se la variabile `name` ha un qualche valore *truthy* (valore non vuoto):

- se `name` è *truthy*, mostra la stringa `<h1>Hello {{ name }}!</h1>`, sostituendo `{{ name }}` con il valore contenuto in `name`;
- se `name` è *falsy*, mostra la stringa generica `<h1>Hello, World!</h1>`.

L'istruzione `endif` è necessaria per comunicare che la struttura di controllo `if..else` è terminata.

ESEMPIO: `templates/hello.html`

```html
<!doctype html>
<title>Hello from Flask</title>
{% if name %}
  <h1>Hello {{ name }}!</h1>
{% else %}
  <h1>Hello, World!</h1>
{% endif %}
```

Per ora ricordiamo che:

- Usiamo `{% ... %}` per includere delle istruzioni o dichiarazioni.

- Usiamo `{{ ... }}` per includere espressioni da stampare nel template renderizzato

## Rispondere con un template Jinja renderizzato

Per rispondere ad una richiesta usando un template, dobbiamo prima renderizzarlo.

Per fare questo usiamo la funzione `render_template`, a cui passiamo il nome del file HTML che vogliamo renderizzare, contenuto nella cartella `/templates` e i nomi, ovvero le variabili che sono richieste all'interno del template.

Nel caso del nostro `hello.html`, l'unica variabile richiesta è `name`, dunque passeremo anche un argomento `name=...`.

```python
@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)
```

# HTTP status codes

Quando viene effettuata una richiesta dal browser, il server può rispondere in diversi modi, ma ciascuna risposta è sempre associata a un relativo corice.

Questi codici li potete vedere nell log di Flask, visibile nel terminale di VS Code, oppure nella sezione "Network" o "Rete" degli strumenti per sviluppatori all'interno del browser.

Il significato primario dei codici di risposta è determinato dalla prima cifra, secondo questo schema:

- `1xx`: *information* - La richiesta è stata ricevuta, proceda chi deve procedere, attenda chi deve attendere.
- `2xx`: *success* - La richiesta è stata ricevuta con successo, compresa ed accettata.
- `3xx`: *redirect* - Il client deve eseguire ulteriori azioni per completare la richiesta.
- `4xx`: *client error* - La richiesta contiene una sintassi errata o non può essere soddisfatta.
- `5xx`: *server error* - Il server non è riuscito a soddisfare una richiesta apparentemente valida.

In generale vale questa regola:

- `1xx`, `2xx`, `3xx` &rarr; NESSUN PROBLEMA
- `4xx`, `5xx` &rarr; C'È QUALCHE PROBLEMA

Per avere un'idea dei loro significati, ecco una lista dei più comuni codici che potrete riscontrare:

- `200` OK
- `301` Moved Permanently
- `302` Found (Moved temporarily)
- `304` Not Modified
- `400` Bad Request
- `401` Unauthorized
- `403` Forbidden
- `404` Not Found
- `500` Internal Server Error
- `503` Service Unavailable
- `501` Not Implemented

Per approfondire il significato di questi codici, potete leggere la pagina di Wikipedia sui [Codici di stato HTTP](https://it.wikipedia.org/wiki/Codici_di_stato_HTTP).

# Headers

Sia la richiesta sia la risposta oltre che a un "contenuto" vengono accompagnati anche da un cosiddetto _**header**_, ovvero una "intestazione".

In parole povere un [header HTTP](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields) è una sorta di bolla di accompagnamento del messaggio, che contiene dei dati utili ai fini della corretta comunicazione tra client e server sotto forma di coppie *campo* (chiave) e *valore*.

Più precisamente, *campi* dell'header HTTP sono un elenco di parole chiave e valori sotto forma di stringhe, le quali sono inviate e ricevute dal client e dal server ad ogni richiesta e risposta HTTP. Queste intestazioni sono solitamente invisibili all'utente e vengono elaborate e loggate solo dalle rispettive applicazioni del server e del client. Definiscono il modo in cui la connessione e le informazioni inviate/ricevute attraverso essa devono essere interpretate e gestite.

# Bookmarklet

(note di cultura generale)

Un *bookmarklet* è un piccolo programma JavaScript che può essere memorizzato come un normale URL all'interno dei segnalibri (bookmark in inglese).

Per esempio, create un nuovo segnalibro nel vostro browser e incollate il seguente codice Jacascript al posto dell'URL:

```javascript
javascript:(function(){
  q=document.getSelection();
  if(!q) q=prompt('Wikipedia:');
  if(q) location.href='http://it.wikipedia.org/w/wiki.phtml?search=' + escape(q);
})()
```

Ora, dopo aver selezionato una parola in una qualunque pagina web, potrete successivamente cliccare sul questo bookmark per avviare automaticamente una ricerca su Wikipedia per la parola selezionata.